### 卷积层
卷积层试图将神经网络中的每一小块进行更加深入的分析从而得到抽象程度更高的特征。一般来说，通过卷积层处理过的节点矩阵会变得更深。

### 池化层
不会改变三维矩阵的深度，但是可以缩小矩阵的大小。通过池化层可以进一步缩小最后全连接层中节点的个数，从而达到减少神经网络中参数的目的。

同时，使用池化层既可以加快计算速度，也有防止过拟合问题的作用。

卷积层使用的过滤器是横跨整个深度的，而池化层使用的过滤器只影响一个深度上的节点。所以，池化层的过滤器除了在长宽两个维度移动外，还需要在深度整个维度移动。

#### 1. 卷积样例

In [1]:
import tensorflow as tf

In [2]:
# input输入图像是一个四维矩阵
# 第1维是第X个图像
# 第2第3维是长度宽度为32*32
# 第4维是RGB颜色通道数为3的矩阵
input = tf.placeholder(
    name='input',
    shape=[None, 32, 32, 3], 
    dtype=tf.float32
)

# filter是一个四维矩阵
# 第1第2维是过滤器尺寸width和height（5*5），
# 第3维当前层深度（3）
# 第4维是过滤器深度（16）
# 初始化为正态分布的随机变量
filter_weight = tf.get_variable(
    name='weights', 
    shape=[5, 5, 3, 16], 
    initializer=tf.truncated_normal_initializer(stddev=0.1)
)

# biases偏置量是一个一维矩阵
# 第1维是偏置量的深度（16）
# 初始化为数值都是0.1的一维矩阵
biases = tf.get_variable(
    name='biases', 
    shape=[16], 
    initializer=tf.constant_initializer(0.1)
)

# tf.nn.conv2d的strides步长参数
# strides步长是一个四维矩阵
# 第1维和第4维必须为1，因为卷积层的步长只对矩阵的长宽有效
# 第2维和第3维是长和宽的步长
# 
# tf.nn.conv2d的padding参数只提供两种SAME以及VAILD
# SAME表示添加0填充
# VAILD表示不添加填充
conv = tf.nn.conv2d(
    input=input, 
    filter=filter_weight, 
    strides=[1,1,1,1], 
    padding='SAME'
)

# tf.nn.bias_add提供了一个方便的函数给节点添加偏置量（注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置量）
# 例如下一层神经网络的大小为2*2，但是偏置量只有一个数（因为深度为1），而2*2矩阵的每一个项都需要加上这个偏置量。
bias = tf.nn.bias_add(value=conv, bias=biases)

# 把计算结果通过relu激活函数进行去线性化
actived_conv = tf.nn.relu(features=bias)


#### 2. 池化样例

In [3]:
# 池化有两种常用的池化
# tf.nn.max_pool最大值池化
# tf.nn.avg_pool均值池化

# ksize为池化层过滤器filter的尺寸，过滤器的头尾两个维度必须等于1
# 实际中使用最多的尺寸是[1,2,2,1]或者[1,3,3,1]
# strides为池化层过滤器步长
pool = tf.nn.max_pool(
    value=actived_conv, 
    ksize=[1,3,3,1], 
    strides=[1,2,2,1], 
    padding='SAME'
)


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(biases))
    print(sess.run(filter_weight))

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[[[[ 0.16086896 -0.08590709  0.01129152 ...  0.0465188  -0.1285064
     0.01130528]
   [-0.13668458 -0.00721074  0.02542484 ...  0.11942949 -0.05869758
     0.08522835]
   [-0.17662907  0.12752213  0.02217315 ... -0.0087734   0.03193136
     0.07070177]]

  [[ 0.07970516  0.00736356  0.04514283 ... -0.1376996   0.11277846
    -0.0463818 ]
   [ 0.00376207 -0.10573174 -0.00382965 ...  0.05850343 -0.13952698
    -0.03884216]
   [ 0.02584567 -0.05464004  0.17732565 ... -0.08232547  0.06970467
     0.08167764]]

  [[ 0.0156565   0.03449583 -0.03315546 ...  0.02672984  0.04649135
    -0.05720701]
   [-0.09376765  0.16665173  0.05982022 ...  0.00117586 -0.04577885
     0.06438689]
   [ 0.09290422 -0.04797619 -0.03077307 ... -0.04613908  0.00763629
    -0.1285299 ]]

  [[-0.00819756 -0.13868015 -0.09443014 ... -0.1005367   0.06331839
    -0.05652886]
   [ 0.03086447 -0.06538675 -0.00331641 ... -0.14971703 -0.00526609
    -0.0153